In [1]:
!pip install beautifulsoup4

Importing libraries

In [2]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
print("Libraries imported!")

Libraries imported!


Load data

In [3]:
data = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M").text
wikidata = BeautifulSoup(data, 'html.parser')

In [4]:
#wikidata

Transform the data into dataframe

In [5]:
body = wikidata.find('tbody')
#body

In [6]:
rows = body.select('tr')
row = [r.get_text() for r in rows]
df = pd.DataFrame(row)
df.head()

,0
0,\nPostal Code\n\nBorough\n\nNeighbourhood\n
1,\nM1A\n\nNot assigned\n\nNot assigned\n
2,\nM2A\n\nNot assigned\n\nNot assigned\n
3,\nM3A\n\nNorth York\n\nParkwoods\n
4,\nM4A\n\nNorth York\n\nVictoria Village\n


In [7]:
df = df[0].str.split('\n', expand=True)
df.head()

,0,1,2,3,4,5,6
0,,Postal Code,,Borough,,Neighbourhood,
1,,M1A,,Not assigned,,Not assigned,
2,,M2A,,Not assigned,,Not assigned,
3,,M3A,,North York,,Parkwoods,
4,,M4A,,North York,,Victoria Village,


In [8]:
df = df.rename(columns=df.iloc[0])
df.head()

,,Postal Code,,Borough,,Neighbourhood,
0,,Postal Code,,Borough,,Neighbourhood,
1,,M1A,,Not assigned,,Not assigned,
2,,M2A,,Not assigned,,Not assigned,
3,,M3A,,North York,,Parkwoods,
4,,M4A,,North York,,Victoria Village,


In [9]:
df = df.drop(df.index[0])
df.head()

,,Postal Code,,Borough,,Neighbourhood,
1,,M1A,,Not assigned,,Not assigned,
2,,M2A,,Not assigned,,Not assigned,
3,,M3A,,North York,,Parkwoods,
4,,M4A,,North York,,Victoria Village,
5,,M5A,,Downtown Toronto,,"Regent Park, Harbourfront",


Ignore cells with a borough that is Not assigned.

In [10]:
df = df[df.Borough != 'Not assigned']
df.head()

,,Postal Code,,Borough,,Neighbourhood,
3,,M3A,,North York,,Parkwoods,
4,,M4A,,North York,,Victoria Village,
5,,M5A,,Downtown Toronto,,"Regent Park, Harbourfront",
6,,M6A,,North York,,"Lawrence Manor, Lawrence Heights",
7,,M7A,,Downtown Toronto,,"Queen's Park, Ontario Provincial Government",


More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma.

In [11]:
df = df.groupby(['Postal Code', 'Borough'], sort = False).agg(','.join)
df.reset_index(inplace = True)
df.head()

,Postal Code,Borough,,,,Neighbourhood,
0,M3A,North York,,,,Parkwoods,
1,M4A,North York,,,,Victoria Village,
2,M5A,Downtown Toronto,,,,"Regent Park, Harbourfront",
3,M6A,North York,,,,"Lawrence Manor, Lawrence Heights",
4,M7A,Downtown Toronto,,,,"Queen's Park, Ontario Provincial Government",


If a cell has a borough but a Not assigned  neighborhood, then the neighborhood will be the same as the borough.

In [12]:
df[df.Neighbourhood == 'Not assigned'] = df[['Borough']]
df.head()

,Postal Code,Borough,,,,Neighbourhood,
0,M3A,North York,,,,Parkwoods,
1,M4A,North York,,,,Victoria Village,
2,M5A,Downtown Toronto,,,,"Regent Park, Harbourfront",
3,M6A,North York,,,,"Lawrence Manor, Lawrence Heights",
4,M7A,Downtown Toronto,,,,"Queen's Park, Ontario Provincial Government",


Shape

In [13]:
df.shape

(103, 7)

Load the csv file that has the geographical coordinates of each postal code into dataframe_geo

In [14]:
df_geo = pd.read_csv("http://cocl.us/Geospatial_data")
df_geo.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


Merge two dataframes

In [15]:
df_l = pd.merge(df, df_geo, on='Postal Code')
df_l.head()

,Postal Code,Borough,,,,Neighbourhood,,Latitude,Longitude
0,M3A,North York,,,,Parkwoods,,43.753259,-79.329656
1,M4A,North York,,,,Victoria Village,,43.725882,-79.315572
2,M5A,Downtown Toronto,,,,"Regent Park, Harbourfront",,43.654260,-79.360636
3,M6A,North York,,,,"Lawrence Manor, Lawrence Heights",,43.718518,-79.464763
4,M7A,Downtown Toronto,,,,"Queen's Park, Ontario Provincial Government",,43.662301,-79.389494
